In [ ]:
pip install s3fs

In [ ]:
from kafka import KafkaConsumer
from json import dumps, loads
import json
from s3fs import S3FileSystem
import pandas as pd
import os

In [31]:
consumer = KafkaConsumer(
    'demo_testing2',
    bootstrap_servers = ['54.82.101.96:9092'],
    value_deserializer = lambda x: 
    loads(x.decode('utf-8')))

In [ ]:
s3 = S3FileSystem()

def save_to_s3(count, i):
    with s3.open("s3://kafka-stock-market-analysis-data-abhishek/stock_market_{}.json".format(count), 'w') as file:
        json.dump(i.value, file)

In [ ]:
df = pd.DataFrame()

for count, i in enumerate(consumer):
    include_headers = not os.path.isfile('stock_streaming_data.csv')
    record_received = pd.DataFrame([i.value])
    df = df._append(record_received, ignore_index = True)
    # df = pd.concat([df, record_received], ignore_index= True)

    # saving json record to S3 bucket: 
    save_to_s3(count,i)

    # saving to csv using batch of 10 records
    if len(df) >= 10:
        print(df.head())
        df.to_csv('stock_streaming_data.csv', mode = 'a', index= False, header = include_headers)
        df = pd.DataFrame()